Sample

## Load pre-trained EPiC-FM models

In [1]:
from DynGenModels.configs.registered_experiments import Config_JetClass_EPiC_CondFlowMatch
from DynGenModels.models.experiment import Experiment
from DynGenModels.datamodules.jetclass.dataprocess import PostProcessJetClassData as PostProcessor
from utils import results_plots 

- time emebdding: Gaussian
- dim_time_emebed = 12

In [2]:
path = '../../results/qcd_to_top_gauss.ConditionalFlowMatching.EPiC.2024.02.16_20h19'
cfm = Experiment(Config_JetClass_EPiC_CondFlowMatch, path=path, DEVICE='cuda:2')
cfm.load(model='best')

cfm.generate_samples(cfm.dataset.source_preprocess[:10000], PostProcessor=PostProcessor)
results_plots(cfm, save_dir=None, features=[r'$p^{\rm rel}_T$', r'$\Delta\eta$', r'$\Delta\phi$'], figsize=(12,3.5))

+---------------------+----------------------------------+
| Parameters          | Values                           |
+---------------------+----------------------------------+
| NAME                | qcd_to_top_gauss                 |
| DATASET             | jetclass                         |
| DATA_SOURCE         | qcd                              |
| DATA_TARGET         | top                              |
| NUM_CONSTITUENTS    | 30                               |
| FEATURES            | ['pt_rel', 'eta_rel', 'phi_rel'] |
| PREPROCESS          | ['standardize']                  |
| DIM_INPUT           | 3                                |
| DEVICE              | cuda:3                           |
| OPTIMIZER           | Adam                             |
| LR                  | 0.0001                           |
| WEIGHT_DECAY        | 5e-05                            |
| OPTIMIZER_BETAS     | [0.9, 0.999]                     |
| OPTIMIZER_EPS       | 1e-08                           

2024-02-16 21:01:26.119236: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 21:01:26.168724: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 21:01:26.169853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-16 21:01:26.924013: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO: building dataloaders...
INFO: train/val/test split ratios: 0.8/0.2/0.0
INFO: train size: 80000, validation size: 20000, testing sizes: 0
INFO: neural ODE solver with midpoint method and steps=200


  0%|          | 0/10 [00:00<?, ?it/s]

/home/df630/.conda/envs/flow_match_env/lib/python3.9/site-packages/torchdyn/numerics/odeint.py:84: UserWarning: Setting tolerances has no effect on fixed-step methods
  warn("Setting tolerances has no effect on fixed-step methods")


### Generate samples from model:

In [ ]:
from DynGenModels.datamodules.jetclass.dataprocess import PostProcessJetClassData as PostProcessor
cfm.generate_samples(cfm.dataset.source_preprocess[:30000], PostProcessor=PostProcessor)
